In [ ]:
! pip install fastapi nest-asyncio pyngrok uvicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 27.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=a6fd2fc3296d3377cb3ad7547263cebaf70a6e81c08c4688a86c971ace2d731c
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
# Install ngrok authtoken
! ngrok authtoken 2GI8Y8CqUD2ajdbcDk2lOA0wqBv_21nfvEC1LSV7KexZVMNx4

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
%pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=90ff1ef93d202d96497d20dbcb9bb785c02bc69a5dd365cca897b4ea4466e9fd
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
! pip install python-multipart

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.9 MB/s eta 0:00:00


In [ ]:
! pip install pyngrok


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! pip install starlette


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Disconnect the existing tunnel
# ngrok.disconnect(ngrok_tunnel.public_url)

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from pydantic import BaseModel
import pandas as pd
from typing import List

from fastapi import FastAPI, HTTPException, Request, Form, Depends
from fastapi.responses import JSONResponse
from fastapi.encoders import jsonable_encoder

# Import your PySpark ML model and preprocessing functions here
from pyspark.sql import functions as F
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import Imputer
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel

app = FastAPI()

# enable CORS
from fastapi.middleware.cors import CORSMiddleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
    expose_headers=["*"],
)


# Add the following line to mount static files
# app.mount("/static", StaticFiles(directory="static"), name="static")

templates = Jinja2Templates(directory="templates")

@app.get("/", response_class=HTMLResponse)
async def serve_homepage(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})


spark = SparkSession.builder \
    .appName("Fetal Health Prediction") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.python.worker.memory", "2g") \
    .getOrCreate()


data = spark.read.csv("/content/fetal_health.csv", header=True, inferSchema=True)

# Handle missing values (if any)
imputer = Imputer(strategy='mean', inputCols=data.columns, outputCols=data.columns)
data = imputer.fit(data).transform(data)

# Assemble features into a single vector
feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# feature scaling
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

# Create pipeline for preprocessing
pipeline = Pipeline(stages=[assembler, scaler])

# Fit the pipeline and transform the data
preprocessing_pipeline_model = pipeline.fit(data)
preprocessed_data = preprocessing_pipeline_model.transform(data) #Just got this fixed up


# Split the dataset into training and testing sets
train_data, test_data = preprocessed_data.randomSplit([0.8, 0.2], seed=42)

# Create a RandomForest model
rf = RandomForestClassifier(labelCol="fetal_health", featuresCol="scaled_features", numTrees=100, seed=42)

# Define input data schema
class InputData(BaseModel):
    baseline_value: float
    accelerations: float
    fetal_movement: float
    uterine_contractions: float
    light_decelerations: float
    severe_decelerations: float
    prolongued_decelerations: float
    abnormal_short_term_variability: float
    mean_value_of_short_term_variability: float
    percentage_of_time_with_abnormal_long_term_variability: float
    mean_value_of_long_term_variability: float
    histogram_width: float
    histogram_min: float
    histogram_max: float
    histogram_number_of_peaks: float
    histogram_number_of_zeroes: float
    histogram_mode: float
    histogram_mean: float
    histogram_median: float
    histogram_variance: float
    histogram_tendency: float

trained_model = None
accuracy_result = None

@app.get("/", response_class=HTMLResponse)
async def serve_homepage(request: Request):
    return templates.TemplateResponse("index.html", {"request": request, "accuracy": accuracy_result})

@app.post("/train")
async def train_model(request: Request):
    global trained_model, accuracy_result
    # Load the dataset, preprocess, and train the model
    from pyspark.ml.classification import RandomForestClassifier
    from pyspark.ml import Pipeline
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator

    # Create a RandomForest model
    rf = RandomForestClassifier(labelCol="fetal_health", featuresCol="scaled_features", numTrees=100, seed=42)

    # Update the pipelines without assembler and scaler stages
    rf_pipeline = Pipeline(stages=[rf])

    # Train the models
    trained_model = rf_pipeline.fit(train_data)

    evaluator = MulticlassClassificationEvaluator(labelCol="fetal_health", predictionCol="prediction")
    rf_accuracy = evaluator.evaluate(trained_model.transform(test_data), {evaluator.metricName: "accuracy"})

    accuracy_result = rf_accuracy

    print("RandomForest Classifier - Accuracy:", rf_accuracy)

    trained_model.write().overwrite().save("fetal_health_prediction_model")

    # return {"status": "Model trained successfully", "accuracy": rf_accuracy}
    return templates.TemplateResponse("result.html", {"request": request, "status": "Model trained successfully: ✅", "accuracy": rf_accuracy})

@app.post("/predict", response_class=JSONResponse)
async def predict(request: Request,
    baseline_value : float = Form(...),
    accelerations: float = Form(...),
    fetal_movement: float = Form(...),
    uterine_contractions: float = Form(...),
    light_decelerations: float = Form(...),
    severe_decelerations: float = Form(...),
    prolongued_decelerations: float = Form(...),
    abnormal_short_term_variability: float = Form(...),
    mean_value_of_short_term_variability: float = Form(...),
    percentage_of_time_with_abnormal_long_term_variability: float = Form(...),
    mean_value_of_long_term_variability: float = Form(...),
    histogram_width: float = Form(...),
    histogram_min: float = Form(...),
    histogram_max: float = Form(...),
    histogram_number_of_peaks: float = Form(...),
    histogram_number_of_zeroes: float = Form(...),
    histogram_mode: float = Form(...),
    histogram_mean: float = Form(...),
    histogram_median: float = Form(...),
    histogram_variance: float = Form(...),
    histogram_tendency: float = Form(...)
):
    global trained_model
    if not trained_model:
        raise HTTPException(status_code=400, detail="Model not trained. Please train the model first.")

    input_data = {
        "baseline_value": baseline_value,
        "accelerations": accelerations,
        "fetal_movement": fetal_movement,
        "uterine_contractions": uterine_contractions,
        "light_decelerations": light_decelerations,
        "severe_decelerations": severe_decelerations,
        "prolongued_decelerations": prolongued_decelerations,
        "abnormal_short_term_variability": abnormal_short_term_variability,
        "mean_value_of_short_term_variability": mean_value_of_short_term_variability,
        "percentage_of_time_with_abnormal_long_term_variability": percentage_of_time_with_abnormal_long_term_variability,
        "mean_value_of_long_term_variability": mean_value_of_long_term_variability,
        "histogram_width": histogram_width,
        "histogram_min": histogram_min,
        "histogram_max": histogram_max,
        "histogram_number_of_peaks": histogram_number_of_peaks,
        "histogram_number_of_zeroes": histogram_number_of_zeroes,
        "histogram_mode": histogram_mode,
        "histogram_mean": histogram_mean,
        "histogram_median": histogram_median,
        "histogram_variance": histogram_variance,
        "histogram_tendency": histogram_tendency
    }

    print("Received form data:", {
        "baseline_value": baseline_value,
        "accelerations": accelerations,
        "fetal_movement": fetal_movement,
        "uterine_contractions": uterine_contractions,
        "light_decelerations": light_decelerations,
        "severe_decelerations": severe_decelerations,
        "prolongued_decelerations": prolongued_decelerations,
        "abnormal_short_term_variability": abnormal_short_term_variability,
        "mean_value_of_short_term_variability": mean_value_of_short_term_variability,
        "percentage_of_time_with_abnormal_long_term_variability": percentage_of_time_with_abnormal_long_term_variability,
        "mean_value_of_long_term_variability": mean_value_of_long_term_variability,
        "histogram_width": histogram_width,
        "histogram_min": histogram_min,
        "histogram_max": histogram_max,
        "histogram_number_of_peaks": histogram_number_of_peaks,
        "histogram_number_of_zeroes": histogram_number_of_zeroes,
        "histogram_mode": histogram_mode,
        "histogram_mean": histogram_mean,
        "histogram_median": histogram_median,
        "histogram_variance": histogram_variance,
        "histogram_tendency": histogram_tendency
    })


    # Preprocess the input data and make predictions
    input_df = pd.DataFrame([input_data])
    input_spark_df = spark.createDataFrame(input_df)

    preprocessed_input = preprocessing_pipeline_model.transform(input_spark_df)

    # Make predictions using the trained model
    predictions = trained_model.transform(preprocessed_input)

    # Collect the predictions
    prediction_result = predictions.collect()

    # Extract the predicted labels
    predicted_labels = [result["prediction"] for result in prediction_result]

    print("Input dataframe:", input_df)

    # return {"predictions": predicted_labels[0]}
    if predicted_labels[0]==1:
      predicted_labels = "Normal ✅"
    elif predicted_labels[0]==2:
      predicted_labels = "Suspicious 🤷‍♂️"
    else:
      predicted_labels = "Pathological 💀"
    return templates.TemplateResponse("predict.html", {"request": request, "prediction_result": predicted_labels})

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [544]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://e763-35-233-186-195.ngrok-free.app
INFO:     58.187.203.196:0 - "GET / HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/protocols/http/h11_impl.py", line 428, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/middleware/proxy_headers.py", line 78, in __call__
    return await self.app(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/fastapi/applications.py", line 276, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/applications.py", line 122, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 184, in __call__
    raise exc
  File "/usr/local/lib/python3.10/dist-packages/starlette/middleware/errors.py", line 162, in __call__
    await self.app(scope, receive, _send)
  File "/usr/local/lib/python3.10/dist-packa

INFO:     58.187.203.196:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     58.187.203.196:0 - "GET / HTTP/1.1" 200 OK
RandomForest Classifier - Accuracy: 0.925531914893617
INFO:     58.187.203.196:0 - "POST /train HTTP/1.1" 200 OK
INFO:     58.187.203.196:0 - "GET / HTTP/1.1" 200 OK
Received form data: {'baseline_value': 1231.0, 'accelerations': 12312.0, 'fetal_movement': 121.0, 'uterine_contractions': 1.0, 'light_decelerations': 1.0, 'severe_decelerations': 1.0, 'prolongued_decelerations': 1.0, 'abnormal_short_term_variability': 1.0, 'mean_value_of_short_term_variability': 1.0, 'percentage_of_time_with_abnormal_long_term_variability': 1.0, 'mean_value_of_long_term_variability': 1.0, 'histogram_width': 1.0, 'histogram_min': 1.0, 'histogram_max': 1.0, 'histogram_number_of_peaks': 1.0, 'histogram_number_of_zeroes': 1.0, 'histogram_mode': 1.0, 'histogram_mean': 1.0, 'histogram_median': 1.0, 'histogram_variance': 1.0, 'histogram_tendency': 1.0}
Input dataframe:    baseline_value  acc